In [ ]:
!pip install whois
!pip install pyquery
!pip install tqdm
!pip install interruptingcow
!pip install requests

  Created wheel for whois: filename=whois-0.9.13-py3-none-any.whl size=9609 sha256=34fbd4884859222c1a4b672b26f95af95254ffbf18425ad56a3b85dec4fe0707
  Stored in directory: /root/.cache/pip/wheels/f5/85/a6/2ac688355ea1a85b0a2b9c0243012f3021333b6c65b7305d86
Successfully built whois
  Created wheel for interruptingcow: filename=interruptingcow-0.8-py3-none-any.whl size=5822 sha256=03ca6961cffb03c0f57950edfc3a6047cdc6c2566713e967c68116aa74d7df4d
  Stored in directory: /root/.cache/pip/wheels/fe/83/45/99297454c39c203890ab65c08db879940862778921cc61a416
Successfully built interruptingcow


In [ ]:
import requests
from interruptingcow import timeout
import whois
from datetime import datetime, timezone
import math
import pandas as pd
import numpy as np
from pyquery import PyQuery
from requests import get
import tensorflow as tf
import keras
import pandas as pd
import gensim
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
from sklearn.model_selection import train_test_split
from keras.models import Sequential
from keras.layers import Dense, Conv1D, Flatten, MaxPooling1D, LSTM, Embedding, Input
from sklearn.model_selection import train_test_split
from sklearn.metrics import multilabel_confusion_matrix
from numpy import unique
from tensorflow.keras import layers
from tensorflow.math import confusion_matrix
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import MinMaxScaler

In [ ]:
class UrlFeaturizer(object):
    def __init__(self, url):
        self.url = url
        self.domain = url.split('//')[-1].split('/')[0]
        self.today = datetime.now().replace(tzinfo=None)

        try:
            self.whois = whois.query(self.domain).__dict__
        except:
            self.whois = None

        try:
            self.response = get(self.url)
            self.pq = PyQuery(self.response.text)
        except:
            self.response = None
            self.pq = None

    ## URL string Features
    def entropy(self):
        string = self.url.strip()
        prob = [float(string.count(c)) / len(string) for c in dict.fromkeys(list(string))]
        entropy = sum([(p * math.log(p) / math.log(2.0)) for p in prob])
        return entropy

    def ip(self):
        string = self.url
        flag = False
        if ("." in string):
            elements_array = string.strip().split(".")
            if(len(elements_array) == 4):
                for i in elements_array:
                    if (i.isnumeric() and int(i)>=0 and int(i)<=255):
                        flag=True
                    else:
                        flag=False
                        break
        if flag:
            return 1 
        else:
            return 0

    def numDigits(self):
        digits = [i for i in self.url if i.isdigit()]
        return len(digits)

    def urlLength(self):
        return len(self.url)

    def numParameters(self):
        params = self.url.split('&')
        return len(params) - 1

    def numFragments(self):
        fragments = self.url.split('#')
        return len(fragments) - 1

    def numSubDomains(self):
        subdomains = self.url.split('http')[-1].split('//')[-1].split('/')
        return len(subdomains)-1

    def domainExtension(self):
        ext = self.url.split('.')[-1].split('/')[0]
        return ext

    ## URL domain features
    def hasHttp(self):
        return 'http:' in self.url

    def hasHttps(self):
        return 'https:' in self.url

    def daysSinceRegistration(self):
        if self.whois and self.whois['creation_date']:
            diff = self.today - self.whois['creation_date'].replace(tzinfo=None)
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0

    def daysSinceExpiration(self):
        if self.whois and self.whois['expiration_date']:
            diff = self.whois['expiration_date'].replace(tzinfo=None) - self.today
            diff = str(diff).split(' days')[0]
            return diff
        else:
            return 0
    
     ## URL Page Features
    def bodyLength(self):
        if self.pq is not None:
            return len(self.pq('html').text()) if self.urlIsLive else 0
        else:
            return 0

    def numTitles(self):
        if self.pq is not None:
            titles = ['h{}'.format(i) for i in range(7)]
            titles = [self.pq(i).items() for i in titles]
            return len([item for s in titles for item in s])
        else:
            return 0

    def numImages(self):
        if self.pq is not None:
            return len([i for i in self.pq('img').items()])
        else:
            return 0

    def numLinks(self):
        if self.pq is not None:
            return len([i for i in self.pq('a').items()])
        else:
            return 0

    def scriptLength(self):
        if self.pq is not None:
            return len(self.pq('script').text())
        else:
            return 0

    def specialCharacters(self):
        if self.pq is not None:
            bodyText = self.pq('html').text()
            schars = [i for i in bodyText if not i.isdigit() and not i.isalpha()]
            return len(schars)
        else:
            return 0

    def scriptToSpecialCharsRatio(self):
        v = self.specialCharacters()
        if self.pq is not None and v!=0:
            sscr = self.scriptLength()/v
        else:
            sscr = 0
        return sscr

    def scriptTobodyRatio(self):
        v = self.bodyLength()
        if self.pq is not None and v!=0:
            sbr = self.scriptLength()/v
        else:
            sbr = 0
        return sbr

    def bodyToSpecialCharRatio(self):
        v = self.bodyLength()
        if self.pq is not None and v!=0:
            bscr = self.specialCharacters()/v
        else:
            bscr = 0
        return bscr

    def urlIsLive(self):
        return self.response == 200

    def run(self):
        data = {}
        data['entropy'] = self.entropy()
        data['numDigits'] = self.numDigits()
        data['urlLength'] = self.urlLength()
        data['numParams'] = self.numParameters()
        data['hasHttp'] = self.hasHttp()
        data['hasHttps'] = self.hasHttps()
        data['urlIsLive'] = self.urlIsLive()
        data['bodyLength'] = self.bodyLength()
        data['numTitles'] = self.numTitles()
        data['numImages'] = self.numImages()
        data['numLinks'] = self.numLinks()
        data['scriptLength'] = self.scriptLength()
        data['specialChars'] = self.specialCharacters()
        data['ext'] = self.domainExtension()
        data['dsr'] = self.daysSinceRegistration()
        data['dse'] = self.daysSinceExpiration()
        data['sscr'] = self.scriptToSpecialCharsRatio()
        data['sbr'] = self.scriptTobodyRatio()
        data['bscr'] = self.bodyToSpecialCharRatio()
        data['num_%20'] = self.url.count("%20")
        data['num_@'] = self.url.count("@")
        data['has_ip'] = self.ip()
    
        return data

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
l = ['/content/drive/MyDrive/feature_data.csv']

# emp = UrlFeaturizer("").run().keys()

# A = pd.DataFrame(columns = emp)
# t=[]
# for j in l:
#     print(j)
#     d=pd.read_csv(j,header=None).to_numpy().flatten()
#     print(d)
#     #for i in d:
#     for i in tqdm(d):
#         try: 
#             with timeout(30, exception = RuntimeError):  
#                 temp=UrlFeaturizer(i).run()
#                 temp["File"]=j.split(".")[0]
#                 t.append(temp)
#         except RuntimeError: 
#             pass 
# A=A.append(t)
#os.chdir('../')
#A.to_csv("features12.csv")

data = pd.read_csv(l[0], index_col=0)
data.head(5)

/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,File,bodyLength,bscr,dse,dsr,entropy,ext,hasHttp,hasHttps,has_ip,numDigits,numImages,numLinks,numParams,numTitles,num_%20,num_@,sbr,scriptLength,specialChars,sscr,urlIsLive,urlLength
0,spam_dataset,31,0.064516,0,0,-4.708307,uk,True,False,0,27,1,0,0,0,0,0,0.000000,0,2,0.000000,False,81
1,spam_dataset,64790,0.306050,0,0,-4.576882,html,True,False,0,12,6,183,0,19,0,0,0.923800,59853,19829,3.018458,False,58
2,spam_dataset,757,0.195509,0,0,-4.205536,html,True,False,0,0,6,8,0,1,0,0,0.000000,0,148,0.000000,False,46
3,spam_dataset,266310,0.230915,0,0,-4.550931,html,True,False,0,13,3,831,0,648,0,0,0.312647,83261,61495,1.353947,False,52
4,spam_dataset,40,0.200000,0,0,-4.689336,php?cPath=193_195_197&products_id=1395&op=list,True,False,0,14,0,0,2,1,0,0,0.000000,0,8,0.000000,False,83


In [ ]:
# feature extractor with any link
features = UrlFeaturizer("https://classroom.google.com/u/1/h").run()
print(features)

{'entropy': -3.8529284750433668, 'numDigits': 1, 'urlLength': 34, 'numParams': 0, 'hasHttp': False, 'hasHttps': True, 'urlIsLive': False, 'bodyLength': 82709, 'numTitles': 2, 'numImages': 1, 'numLinks': 7, 'scriptLength': 56534, 'specialChars': 23560, 'ext': 'com', 'dsr': 0, 'dse': 0, 'sscr': 2.3995755517826827, 'sbr': 0.6835289992624745, 'bscr': 0.28485412712038594, 'num_%20': 0, 'num_@': 0, 'has_ip': 0}


In [ ]:
data = pd.read_csv('/content/drive/MyDrive/feature_data.csv')
data.replace(True,1,inplace = True)    
data.replace(False,0,inplace = True)

# remove the bad data point with long string
data = data[data.dse.str.len()<10]   
data = data[data.dsr.str.len()<10] 
data = data[data.ext.str.len()<10] 

data.ext = pd.Categorical(data.ext).codes
print(data.ext.head(5))

y = data["File"]  
data = data.drop(columns = ["File", "Unnamed: 0"])
encoder = LabelEncoder()    
encoder.fit(y)    
Y = encoder.transform(y)  
   
scaler = MinMaxScaler(feature_range=(0, 1))    

X = pd.DataFrame(scaler.fit_transform(data))


/usr/local/lib/python3.7/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (4,5) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


0    471
1    250
2    250
3    250
5    471
Name: ext, dtype: int16


In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from keras.utils.np_utils import to_categorical

In [ ]:
input_dim = len(data.columns)

model = Sequential()    
model.add(Dense(256, input_dim = input_dim , activation = 'relu'))    
model.add(Dense(128, activation = 'relu'))    
model.add(Dense(64, activation = 'relu'))    
model.add(Dense(32, activation = 'relu'))    
model.add(Dense(16, activation = 'relu'))    
model.add(Dense(3, activation = 'softmax'))
model.compile(loss = 'categorical_crossentropy' ,optimizer='adam' , metrics = ['accuracy', 'Recall', 'Precision'] )

model.summary()

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense_6 (Dense)             (None, 256)               5888      
                                                                 
 dense_7 (Dense)             (None, 128)               32896     
                                                                 
 dense_8 (Dense)             (None, 64)                8256      
                                                                 
 dense_9 (Dense)             (None, 32)                2080      
                                                                 
 dense_10 (Dense)            (None, 16)                528       
                                                                 
 dense_11 (Dense)            (None, 3)                 51        
                                                                 
Total params: 49,699
Trainable params: 49,699
Non-trai

In [ ]:
model.fit(X_train, to_categorical(y_train), epochs = 500) 

Epoch 1/500
624/624 [==============================] - 4s 4ms/step - loss: 0.5528 - accuracy: 0.7550 - recall: 0.6979 - precision: 0.7938
Epoch 2/500
624/624 [==============================] - 2s 3ms/step - loss: 0.3424 - accuracy: 0.8603 - recall: 0.8539 - precision: 0.8654
Epoch 3/500
624/624 [==============================] - 2s 3ms/step - loss: 0.2767 - accuracy: 0.8915 - recall: 0.8870 - precision: 0.8958
Epoch 4/500
624/624 [==============================] - 2s 3ms/step - loss: 0.2423 - accuracy: 0.9049 - recall: 0.9004 - precision: 0.9089
Epoch 5/500
624/624 [==============================] - 2s 3ms/step - loss: 0.2220 - accuracy: 0.9149 - recall: 0.9111 - precision: 0.9189
Epoch 6/500
624/624 [==============================] - 2s 3ms/step - loss: 0.2091 - accuracy: 0.9217 - recall: 0.9187 - precision: 0.9246
Epoch 7/500
624/624 [==============================] - 2s 3ms/step - loss: 0.1935 - accuracy: 0.9280 - recall: 0.9251 - precision: 0.9306
Epoch 8/500
624/624 [=============

In [ ]:
model.evaluate(X_test, to_categorical(y_test))

156/156 [==============================] - 1s 2ms/step - loss: 0.2370 - accuracy: 0.9679 - recall: 0.9677 - precision: 0.9681


[0.23703671991825104,
 0.9679294228553772,
 0.9677290320396423,
 0.9681171178817749]

In [ ]:
# same model to test validation split

input_dim = len(data.columns)

model_validation = Sequential()    
model_validation.add(Dense(256, input_dim = input_dim , activation = 'relu'))    
model_validation.add(Dense(128, activation = 'relu'))    
model_validation.add(Dense(64, activation = 'relu'))    
model_validation.add(Dense(32, activation = 'relu'))    
model_validation.add(Dense(16, activation = 'relu'))    
model_validation.add(Dense(3, activation = 'softmax'))
model_validation.compile(loss = 'categorical_crossentropy' ,optimizer='adam' , metrics = ['accuracy', 'Recall', 'Precision'] )

model_validation.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               5888      
                                                                 
 dense_1 (Dense)             (None, 128)               32896     
                                                                 
 dense_2 (Dense)             (None, 64)                8256      
                                                                 
 dense_3 (Dense)             (None, 32)                2080      
                                                                 
 dense_4 (Dense)             (None, 16)                528       
                                                                 
 dense_5 (Dense)             (None, 3)                 51        
                                                                 
Total params: 49,699
Trainable params: 49,699
Non-traina

In [ ]:
model_validation.fit(X_train,to_categorical(y_train), epochs = 300, validation_split=0.2)

Epoch 1/300
499/499 [==============================] - 2s 4ms/step - loss: 0.0454 - accuracy: 0.9820 - recall: 0.9820 - precision: 0.9821 - val_loss: 0.2011 - val_accuracy: 0.9612 - val_recall: 0.9612 - val_precision: 0.9612
Epoch 2/300
499/499 [==============================] - 2s 4ms/step - loss: 0.0478 - accuracy: 0.9810 - recall: 0.9809 - precision: 0.9811 - val_loss: 0.1833 - val_accuracy: 0.9662 - val_recall: 0.9659 - val_precision: 0.9662
Epoch 3/300
499/499 [==============================] - 2s 4ms/step - loss: 0.0440 - accuracy: 0.9829 - recall: 0.9829 - precision: 0.9829 - val_loss: 0.2044 - val_accuracy: 0.9612 - val_recall: 0.9607 - val_precision: 0.9614
Epoch 4/300
499/499 [==============================] - 2s 4ms/step - loss: 0.0552 - accuracy: 0.9786 - recall: 0.9786 - precision: 0.9788 - val_loss: 0.1933 - val_accuracy: 0.9632 - val_recall: 0.9632 - val_precision: 0.9634
Epoch 5/300
499/499 [==============================] - 2s 4ms/step - loss: 0.0466 - accuracy: 0.9817

In [ ]:
model_validation.evaluate(X_test, to_categorical(y_test))

156/156 [==============================] - 0s 2ms/step - loss: 0.2642 - accuracy: 0.9671 - recall: 0.9671 - precision: 0.9671


[0.2642240524291992,
 0.9671276807785034,
 0.9671276807785034,
 0.9671276807785034]